In [30]:
from multiprocessing import Process
from pathlib import Path
import os
import pandas as pd

In [9]:
def info(title):
    print(title)
    print('module name:', __name__)
    print('parent process:', os.getppid())
    print('process id:', os.getpid())

def f(name):
    info('function f')
    print('hello', name)

if __name__ == '__main__':
    info('main line')
    p = Process(target=f, args=('bob',))
    p.start()
    p.join()
    print('after join')

main line
module name: __main__
parent process: 5292
process id: 5512
after join
function f
module name: __main__
parent process: 5512
process id: 18001
hello bob


In [41]:
def freqs(in_file, out_path):

    freqs = {}
    
    file_name = os.path.basename(in_file)

    with open(in_file, 'r') as in_file:
        text = in_file.readlines()

        for line in text:
            line = line.split()
            for word in line:
                if word not in freqs.keys():
                    freqs[word] = 1
                else:
                    freqs[word] += 1

    freqs = dict(sorted(freqs.items(), key=lambda item: item[1], reverse=True))
    df = pd.DataFrame(freqs.items(), columns=['Lemma', 'Count'])

    with open(f"{out_path}/freqs_{file_name}.tsv", 'w+') as out_file:
        df.to_csv(out_file, sep='\t')

    return freqs

def split(in_file, out_path, num_splits):
    
    file_name = os.path.basename(in_file)
    
    with open(in_file, 'r') as in_file:
        
        data = in_file.readlines()
        num_lines = len(data)
        split_len = num_lines%num_splits
        
        text_data = []
        file_count = 0

        for sample in data:
            sample = sample.replace('\n', '')
            text_data.append(sample)

            save_path = f'{out_path}/{file_name}_parts/{file_name}_{file_count}.txt'
            print(save_path)
            print(len(text_data))
            print(split_len)
            if len(text_data) == split_len:
                
                with open(save_path, 'w+', encoding='utf-8') as fp:
                    fp.write('\n'.join(text_data))
                text_data = []
                file_count += 1
    
    paths = [str(x) for x in Path(f'{out_path}/{file_name}_parts').glob('**/*.txt')]
    return paths

def find_rare(freqs, threshold):

    rare = []
    for key,value in freqs.items():
        if value <= threshold:
            rare.append(key)
    
    return rare

def filter_rare(in_file, out_path, freqs_file, threshold):

    print(f"Filtering {len(rare)} words with freq lower than {threshold}")

    with open(f'./data/days_{self.year}/processed/days_{self.year}_sentencesL_prepost.txt', 'r') as in_file:
        text = in_file.readlines()

        if os.path.isfile(f'./data/days_{self.year}/processed/days_{self.year}_sentencesL_post.txt'):
            os.remove(f'./data/days_{self.year}/processed/days_{self.year}_sentencesL_post.txt')

        with open(f'./data/days_{self.year}/processed/days_{self.year}_sentencesL_post.txt', 'a+') as out_file:
            i = 0
            for line in text:
                start = time.time()
                print(f"Filtering line {i} out of {len(text)}")
                for word in line.split():
                    if (word in rare): # and (word not in unique): # Too long
                        line = re.sub(word, '', line)
                print(line, file=out_file)
                end = time.time()
                print(time.strftime("%H:%M:%S", time.gmtime(end-start)))
                if i%10 == 0:
                    clear_output(wait=True)
                    print(f"Filtering {len(rare)} words with freq lower than {threshold}")
                i+=1

def run(in_file, out_path, num_splits, threshold):
    freqs_dict = freqs(in_file, out_path)
    rare = find_rare(freqs_dict, threshold)
    splits = split(in_file, out_path, num_splits)


In [42]:
run('/home/edo/projects/usage_change_ITA/data/days_2020/processed/days_2020_sentencesL_prepost.txt', './test', 12, 10)

./test/days_2020_sentencesL_prepost.txt_parts/days_2020_sentencesL_prepost.txt_0.txt
1
8
./test/days_2020_sentencesL_prepost.txt_parts/days_2020_sentencesL_prepost.txt_0.txt
2
8
./test/days_2020_sentencesL_prepost.txt_parts/days_2020_sentencesL_prepost.txt_0.txt
3
8
./test/days_2020_sentencesL_prepost.txt_parts/days_2020_sentencesL_prepost.txt_0.txt
4
8
./test/days_2020_sentencesL_prepost.txt_parts/days_2020_sentencesL_prepost.txt_0.txt
5
8
./test/days_2020_sentencesL_prepost.txt_parts/days_2020_sentencesL_prepost.txt_0.txt
6
8
./test/days_2020_sentencesL_prepost.txt_parts/days_2020_sentencesL_prepost.txt_0.txt
7
8
./test/days_2020_sentencesL_prepost.txt_parts/days_2020_sentencesL_prepost.txt_0.txt
8
8


FileNotFoundError: [Errno 2] No such file or directory: './test/days_2020_sentencesL_prepost.txt_parts/days_2020_sentencesL_prepost.txt_0.txt'